In [1]:
import asyncio, nest_asyncio, re, datetime
from pyppeteer import launch
import pandas as pd

nest_asyncio.apply()

In [4]:
import asyncio
import nest_asyncio
from pyppeteer import launch

def extract_rarity_and_card_name(content):
    pattern = r'(UR|AR|SR|SAR)\s+([^\s\n]+)'
    matches = re.findall(pattern, content)
    return matches[0][0], matches[0][1]

def extract_card_index(content):
    pattern = r'\d{3}/\d{3}'
    match = re.search(pattern, content)
    if match:
        return match.group()
    return None

def extract_card_price(content):
    pattern = r'\d{1,3}(?:,\d{3})* 円'
    match = re.search(pattern, content)
    if match:
        price_str = match.group()
        return int(price_str.replace(',', '').replace(' 円', ''))
    return None

async def main(i):
    browser = await launch(headless=True) 
    page = await browser.newPage()

    await page.goto(f'https://yuyu-tei.jp/sell/poc/card/sv09a/{i}')

    await page.waitForSelector('.fw-bold')

    fw_bold_texts = await page.evaluate('''() => {
        const boldElements = document.querySelectorAll('.fw-bold');
        return Array.from(boldElements).map(element => element.innerText).join('\\n');
    }''')
    await browser.close()
    return fw_bold_texts

In [5]:
df = pd.DataFrame(columns=['card_rarity', 'card_name', 'card_index', 'card_price', 'created_time'])


for i in range(10075,10092):
    content = asyncio.run(main(i))
    card_rarity, card_name = extract_rarity_and_card_name(content)
    card_index = extract_card_index(content)
    card_price = extract_card_price(content)
    created_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    df.loc[len(df)] = [card_rarity, card_name, card_index, card_price, created_time]


In [12]:
df.to_csv(f'../data/ptcg/{datetime.datetime.now().strftime("%Y%m%d")}.csv', index=False)